In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Hyperparameter Optimization**

# Keras Tuner- Decide Number of Hidden Layers And Number of Neuron In A Hidden Layer

In [2]:
import tensorflow

In [3]:
print(tensorflow.__version__)

2.6.0


In [4]:
pip install keras-tuner

     |████████████████████████████████| 97 kB 3.2 MB/s 


In [5]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  after removing the cwd from sys.path.


In [6]:
df = pd.read_csv('/content/drive/MyDrive/Deep Learning/Datasets/KNDataSets/Real_Combine.csv')
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [7]:
#independent features
X=df.iloc[:,:-1]
X.head()

,T,TM,Tm,SLP,H,VV,V,VM
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2


In [8]:
#dependent features
y=df.iloc[:,-1]
y.head()

0    219.720833
1    182.187500
2    154.037500
3    223.208333
4    200.645833
Name: PM 2.5, dtype: float64


# **Hyperparameters**

    How many number of hidden layers we should have?
    How many number of neurons we should have in hidden layers?
    Learning Rate

In [9]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):  # 2 - 20 layers in a network
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,   #number of neurons in a layer would be 32 to 512
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear'))   #output layer number of neuron = 1
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [10]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project',
    project_name='Air Quality Index')

In [11]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [12]:
# Train Test Split:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [13]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 04s]
val_mean_absolute_error: nan

Best val_mean_absolute_error So Far: nan
Total elapsed time: 00h 00m 49s
INFO:tensorflow:Oracle triggered exit


In [14]:
tuner.results_summary()

Results summary
Results in project/Air Quality Index
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 15
units_0: 288
units_1: 288
learning_rate: 0.0001
units_2: 32
units_3: 32
units_4: 32
units_5: 32
units_6: 32
units_7: 32
units_8: 32
units_9: 32
units_10: 32
units_11: 32
units_12: 32
units_13: 32
units_14: 32
Score: nan
Trial summary
Hyperparameters:
num_layers: 4
units_0: 448
units_1: 32
learning_rate: 0.0001
units_2: 448
units_3: 128
units_4: 416
units_5: 64
units_6: 448
units_7: 224
units_8: 128
units_9: 160
units_10: 96
units_11: 288
units_12: 256
units_13: 416
units_14: 384
Score: nan
Trial summary
Hyperparameters:
num_layers: 19
units_0: 352
units_1: 224
learning_rate: 0.001
units_2: 448
units_3: 384
units_4: 64
units_5: 416
units_6: 352
units_7: 160
units_8: 480
units_9: 384
units_10: 352
units_11: 160
units_12: 288
units_13: 416
units_14: 384
units_15: 32
units_16: 32
units_17: 32
units_18: 32
Score: